### 1) Importing Libraries

In [2]:
import sys
import os
sys.path.append(os.path.abspath("../"))
from recsys import *
from generic_preprocessing import *
from IPython.display import HTML

### 2) Importing Data

In [4]:
ratings = pd.read_csv('./ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [14]:
movies = pd.read_csv('./ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


### 3) Preprocessing

#### 3.1) Create interaction matrix

In [10]:
interactions = create_interaction_matrix(df = ratings,
                                         user_col = 'userId',
                                         item_col = 'movieId',
                                         rating_col = 'rating',
                                         threshold = '3')
interactions.shape

(671, 9066)

In [12]:
interactions.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### 3.2) Create User Dict

In [15]:
user_dict = create_user_dict(interactions=interactions)

#### 3.3) Create Item dict

In [16]:
movies_dict = create_item_dict(df = movies,
                               id_col = 'movieId',
                               name_col = 'title')

### 4) Building Matrix Factorization model

In [17]:
mf_model = runMF(interactions = interactions,
                 n_components = 30,
                 loss = 'warp',
                 k = '15',
                 epoch = 30,
                 n_jobs = 4)

### 5) User Recommender

In [31]:
rec_list = sample_recommendation_user(model = mf_model, 
                                      interactions = interactions, 
                                      user_id = 10, 
                                      user_dict = user_dict,
                                      item_dict = movies_dict, 
                                      threshold = 4,
                                      nrec_items = 10)

Known Likes:
1- Usual Suspects, The (1995)
2- Sling Blade (1996)
3- My Own Private Idaho (1991)
4- Sweet Hereafter, The (1997)
5- There's Something About Mary (1998)
6- Runaway Train (1985)
7- Matrix, The (1999)
8- 13th Warrior, The (1999)
9- Hairspray (1988)

 Recommended Items:
1- Star Wars: Episode IV - A New Hope (1977)
2- Alien (1979)
3- Silence of the Lambs, The (1991)
4- Fargo (1996)
5- Aliens (1986)
6- Godfather, The (1972)
7- Star Wars: Episode VI - Return of the Jedi (1983)
8- Saving Private Ryan (1998)
9- Back to the Future (1985)
10- Star Wars: Episode V - The Empire Strikes Back (1980)


### 6) Item-User Recommender

In [37]:
sample_recommendation_item(model = mf_model,
                           interactions = interactions,
                           item_id = 1,
                           user_dict = user_dict,
                           item_dict = movies_dict,
                           number_of_user = 15)

[657, 410, 495, 109, 415, 59, 400, 145, 556, 568, 174, 116, 317, 108, 369]

### 7) Item - Item Recommender

In [38]:
item_item_dist = create_item_emdedding_distance_matrix(model = mf_model,
                                                       interactions = interactions)

In [39]:
rec_list = item_item_recommendation(item_emdedding_distance_matrix = item_item_dist,
                                    item_id = 1,
                                    item_dict = movies_dict,
                                    n_items = 10)

Item of interest :Toy Story (1995)
Item similar to the above item:
1- Mask, The (1994)
2- Forrest Gump (1994)
3- Independence Day (a.k.a. ID4) (1996)
4- Silence of the Lambs, The (1991)
5- Titanic (1997)
6- Jurassic Park (1993)
7- Groundhog Day (1993)
8- Men in Black (a.k.a. MIB) (1997)
9- Star Wars: Episode IV - A New Hope (1977)
10- Fifth Element, The (1997)
